## **Formato del dataset**
```bash
wingbeats_dataset/
├── species_1/
│   ├── audio_1.wav
│   ├── audio_2.wav
│   └── ...
├── species_2/
│   ├── audio_1.wav
│   └── ...
...
|
├── species_6/
│   ├── audio_1.wav
│   └── ...
```


### **Formato de subcarpetas por especie**
```bash
D_x_y_z_a_b_c
│
├── D       = Identificador de dispositivo/grabadora (D = Device)
├── x       = Año 
├── y       = Mes 
├── z       = Día 
├── a       = Hora 
├── b       = Minutos 
└── c       = Segundos
```

## **Celda con variables globales e importaciones**

In [2]:
from pathlib import Path
import librosa  

# Ae_Aegypti, Ae_Albopictus, An_Arabiensis, An_Gambiae, Cu_Pipiens, Cu_Quinquefasciatus
n_classes = 6
classes = {k:i for i,k in enumerate(['An. gambiae', 'C. pipiens', 'C. quinquefasciatus', 'Ae. albopictus', 'An. arabiensis', 'Ae. aegypti'])}
MIN_ELEMENTS = False
PATH = 'Wingbeats/'
SAMPLE_RATE = 8_000

In [3]:
classes

{'An. gambiae': 0,
 'C. pipiens': 1,
 'C. quinquefasciatus': 2,
 'Ae. albopictus': 3,
 'An. arabiensis': 4,
 'Ae. aegypti': 5}

## **Extracion de direccion de todos los audios**

In [4]:

def explore_nested_folders(root_path, min_elementos = True):
    root = Path(root_path)
    main_folders = [f for f in root.iterdir() if f.is_dir()]
    estructura = dict()

    for main_folder in main_folders:
        estructura[main_folder.name] = []
        sub_folders = [f for f in main_folder.iterdir() if f.is_dir()]    
    
        for sub_folder in sub_folders:
            files = [f for f in sub_folder.iterdir() if f.is_file()]
            
            for file in files:
                direccion_audio = PATH + str(file.relative_to(root))
                data,sr = librosa.load(direccion_audio, sr=SAMPLE_RATE)
                estructura[main_folder.name].append(
                    dict(
                        direccionName=direccion_audio,
                        sr=sr,
                        duration=len(data)/sr
                    )
                )
        print('Folder completado')
    
    n_samples_min = min([len(files) for files in estructura.values()])
    if min_elementos:
        estructura = {labels:data[:n_samples_min] for labels, data in estructura.items()}
    return estructura, n_samples_min

#folder_structure, min_samples = explore_nested_folders(PATH, min_elementos=MIN_ELEMENTS)
folder_structure, min_samples = explore_nested_folders(PATH, min_elementos=MIN_ELEMENTS)

Folder completado
Folder completado
Folder completado
Folder completado
Folder completado
Folder completado


## **Datos basicos del dataset extraido**

In [5]:
print('Total audios = ',sum([len(total) for total in folder_structure.values()]))
print('Cantidad de audios por clase\n',[len(total) for total in folder_structure.values()])
print('Ejemplo de direccion de archivo = ', folder_structure[list(folder_structure.keys())[0]][0]['direccionName'])

Total audios =  279466
Cantidad de audios por clase
 [49471, 30415, 74599, 20131, 19297, 85553]
Ejemplo de direccion de archivo =  Wingbeats/An. gambiae/D_17_01_17_11_28_16/F170117_112822_068_G_050.wav


## **Validacon de propiedades de los audios**

In [6]:
def allEqual(value):
    funcSR = lambda j : [folder_structure[list(folder_structure.keys())[j]][i][value] for i in range(min_samples)]
    sameSR = [True for i in range(n_classes) if len(set(funcSR(i))) == 1]
    return all(sameSR)

if allEqual('sr'):
    print('Misma tasa de frecuencia para todos los audios ')
    print('Ejemplo de sr de archivo = ', folder_structure[list(folder_structure.keys())[0]][0]['sr'])
else:
    print('Algun audio con diferente tasa de frecuencia')

if allEqual('duration'):
    print('Misma duracion para todos los audios ')
    print('Duracion de archivo = ', folder_structure[list(folder_structure.keys())[0]][0]['duration'])
else:
    print('Algun audio con diferente duracion')

Misma tasa de frecuencia para todos los audios 
Ejemplo de sr de archivo =  8000
Misma duracion para todos los audios 
Duracion de archivo =  0.625


## **Crear anotaciones de los audios junto con su respectiva etiqueta**
```bash
'An. gambiae': 0
'C. pipiens': 1
'C. quinquefasciatus': 2
'Ae. albopictus': 3
'An. arabiensis': 4
'Ae. aegypti': 5
```

In [7]:
def crearAnotaciones():
    clases = list(folder_structure.keys())
    annotations_path = PATH + 'min_annotations.txt' if MIN_ELEMENTS else PATH + 'annotations.txt'
    files_name = [dataFile['direccionName'] + ',' + str(classes[class_mos]) + '\n' 
                  for class_mos in clases
                  for dataFile in folder_structure[class_mos]]
    with open(annotations_path, 'w', encoding='utf-8') as f:
        f.write("file_path,label\n")
        f.writelines(files_name)
crearAnotaciones()